In [1]:
import requests
from bs4 import BeautifulSoup
import json

searchable_pages = [
    "sol/strain",
    "sol/stress",
    "sol/material_properties",
    "sol/axial_loading",
    "dyn/vectors",
    "dyn/vector_calculus",
    "sta/introduction",
    "sta/cartesian_coordinates",
    "sta/vectors_scalars"
]

page_text = []

for page in searchable_pages:
    url = f"http://localhost:4321/{page}"
    html = requests.get(url).text


    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style", "nav"]):
        script.extract()    # rip it out

    soup.find("div", attrs={"id": "nav_container"}).extract()
    soup.find("h1", attrs={"id": "page_title"}).extract()
    soup.find("div", attrs={"id": "title_bar"}).extract()

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = ' '.join(chunk for chunk in chunks if chunk)

    banned_strings = [
        "Derivation +",
        "Solution +"
    ]

    for b in banned_strings:    
        text = text.replace(b, "")

    page_text.append({"title": page.split("/")[1].replace("_", " ").capitalize(),"text": text, 'link': "/" + page})

json.dump(page_text, open("./public/search.json", "w"), indent=2)